In [8]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

import itertools

from py_wake.deficit_models import EddyViscosityDeficitModel, EddyViscosityModel, BastankhahGaussianDeficit
from py_wake import HorizontalGrid

# turbines and sites
from py_wake.site import XRSite, UniformWeibullSite, UniformSite
from py_wake.site._site import Site
from py_wake.site.shear import PowerShear
from py_wake.wind_turbines import WindTurbine
from py_wake.wind_turbines.generic_wind_turbines import GenericWindTurbine
from py_wake.examples.data.hornsrev1 import Hornsrev1Site, V80
from py_wake.examples.data.iea37 import IEA37Site, IEA37_WindTurbines
from py_wake.examples.data.ParqueFicticio import ParqueFicticioSite
from py_wake.examples.data.dtu10mw import DTU10MW

from py_wake.wind_farm_models.wind_farm_model import SimulationResult
from py_wake.wind_farm_models import PropagateDownwind
from py_wake.wind_turbines import WindTurbine, WindTurbines
from py_wake.wind_turbines.power_ct_functions import PowerCtTabular
from py_wake.utils.generic_power_ct_curves import standard_power_ct_curve

import metrics
import utils


#TODO move all these functions in an "utils" file
def my_arange(start, end, step):
    '''Function for np.arange(start, stop, step) without creating problems with the float representations'''
    factor = 10
    while factor*step < 1:
        factor *= 10
    return np.arange(start*factor, end*factor, step*factor) / factor

# Preparation of the parameters for the data generation

## Site (to set turbulence intensity TI and...?)
For a given position, reference wind speed (WSref) and wind direction (WDref), Site provides the local wind condition in terms of wind speed (WS), wind direction (WD), turbulence intensity (TI) and the probability of each combination of wind direction and wind speed. Furthermore, Site is responsible for calculating the down-wind, cross-wind and vertical distance between wind turbines (which in non-flat terrain is different from the straight-line distances).

In [9]:
def get_site(ti: float, ws: int) -> XRSite:
    #return XRSite(ds=xr.Dataset(data_vars={'WS': WS_RANGE, 'P': 1, 'TI': turbulence_intensity}, coords={'wd': WD_RANGE}))
    return UniformSite(ti=ti, ws=ws)

## Turbine (to set thrust coefficient CT)

For a given wind turbine type and effective wind speed (WSeff), the WindTurbine object provides the power and thrust coefficient (CT), as well as the wind turbine hub height (H) and diameter (D).

Initializing a simple WindTurbine
- taking diameter, hub height and power norm from IEA37 as these params affect the simulation;
- default power ct function to set a constant thrust coefficient (CT).

In [10]:
def plot_ct_curve(custom_turbine = None) -> None:
    plt.xlabel('Wind speed [m/s]')
    plt.ylabel('CT [-]')
    wts = WindTurbines.from_WindTurbine_lst([custom_turbine]) #add other elements to this list to compare ct curves
    ws = np.arange(0,30)
    for t in wts.types():
        plt.plot(ws, wts.ct(ws, type=t),'.-', label=wts.name(t))
    plt.legend(loc=1)
    plt.show()

def get_wind_turbine(diameter: int, hub_height: int, power_norm: int,
                     constant_ct: float, ti: float) -> WindTurbine:
    # for power ct function (similar to GenericWindTurbine but putting a constant ct)
    wsp_lst = np.arange(.1, 30, .1) #TODO this parameter decides the number of elements in u, p and ct_lst
    u, p, ct_lst = standard_power_ct_curve(power_norm, diameter, turbulence_intensity=ti, 
                                       constant_ct=constant_ct, wsp_lst=wsp_lst)
    ct_lst = [constant_ct] * len(ct_lst) # make the ct constant
    ct_function = PowerCtTabular(u, p * 1000, 'w', ct_lst, ct_idle=constant_ct)
    return WindTurbine(
        name="AinslieTurbine",
        diameter=diameter,
        hub_height=hub_height,
        power_norm=power_norm,
        powerCtFunction=ct_function
    )

## Discretization
My current reasoning for the discretization:

FOR X RANGE
- the start is diameter*2 to skip the near-wake (careful with the wind direction, if it is within the whole range (0, 360), near-wake intercepted);
- the end is diameter*50 (i.e. DEFAULT_MAXIMUM_WAKE_DISTANCE in PyWake EddyViscosity definition); [#TODO reduce it?]
- the step is parametrized (Javier's thesis trying different values, e.g. diameter/2, diameter/4, ..., diameter/16)

FOR Y RANGE
- since the wind direction remains stable at 270, it is useless to put a big range for y

In [11]:
def get_discretized_grid(diameter: int,
                         x_start_factor: int, x_end_factor: int,
                         y_start_factor: int, y_end_factor: int,
                         grid_step_factor: float) -> HorizontalGrid:
    # TODO check the discretization
    #   - see Javier thesis, appendix B in particular)
    #   - see also datadriven wind turbine wake modelling via probabilistic ML (e.g. Fig. 3) to set these parameters
    x_range = np.arange(diameter*x_start_factor, diameter*x_end_factor, diameter*grid_step_factor)
    y_range = np.arange(diameter*y_start_factor, diameter*y_end_factor, diameter*grid_step_factor)
    return HorizontalGrid(x = x_range, y = y_range)

# Actual data generation

In [12]:
def generate_wake_dataset(model, wind_speed: float, wind_direction: float,
                            wind_diameter: int, turbine_xs: list[int], turbine_ys: list[int],
                            horizontal_grid: HorizontalGrid, wind_turbine: WindTurbine
                            ) -> pd.DataFrame:

    
    # the flow_map creates a warning for near-wake calculations
    sim_res = model(
        x=turbine_xs, y=turbine_ys, # wind turbine positions (setting also wt domain, i.e. the number of turbines)
        wd=wind_direction,          # Wind direction (None for default -> 0-360° in bins of 1°)
        ws=wind_speed,              # Wind speed (None for default -> 3-25 m/s in bins of 1 m/s)
        #yaw=0                      # TODO try to change this parameter?
        #h=None,                    # wind turbine heights (defaults to the heights defined in windTurbines)
        #type=0,                    # Wind turbine types
    )
    flow_map = sim_res.flow_map(horizontal_grid)
    
    # removing h, wd and ws
    h = flow_map['h'].item()
    ti = sim_res.TI.item()
    #print(f"ti in generation: {ti}")
    flow_map = flow_map\
        .sel(h=h)\
        .sel(wd=wind_direction)\
        .sel(ws=wind_speed)
    flow_map = flow_map.drop_vars(["h", "wd", "ws", "WD", "P", "TI", "TI_eff"])

    # adding the input variables
    flow_map["ti"] = xr.DataArray([ti], dims="ti")
    ct = wind_turbine.ct(ws=wind_speed)
    flow_map["ct"] = xr.DataArray([ct], dims="ct")
    # computing the wind_deficit as new data variable
    flow_map["wind_deficit"] = 1 - flow_map["WS_eff"] / flow_map["WS"]
    # scaling x and y according to the diameter
    flow_map["x"] = flow_map["x"] / wind_diameter
    flow_map["y"] = flow_map["y"] / wind_diameter
    flow_map = flow_map.rename({"x": "x:D", "y": "y:D"})

    flow_map = flow_map.astype({'WS_eff': 'float32', 'wind_deficit': 'float32', 'WS': 'int'})
    flow_map = xr.Dataset(flow_map)
    return flow_map

In [13]:
# default parameters
TURBINE_X = [0]
TURBINE_Y = [0]
WIND_DIRECTION = 270 # the wind turbine's yaw angle is always adjusted according to the wind direction, thus it is probably useless to generate data with more wind directions
#WD_RANGE = range(0, 360) # (by default, 0-360° in bins of 1°)

# IEA37 values TODO try different ones?
TURBINE_DIAMETER = 198
TURBINE_HUB_HEIGHT = 119
TURBINE_POWER_NORM = 10000

# discretization factors
X_START_FACTOR = 2
X_END_FACTOR = 50
Y_START_FACTOR = -1 #TODO -2
Y_END_FACTOR = 1 #TODO +2
STEP_FACTOR = 1/8
#TODO put a minimum value for the deficit to delimitate the interesting zone (1/1000)
#TODO xarray netcdf for storing in a more efficient way (including compression)

# parameters for data generation TODO check which right values to use for generation
#WS_RANGE = range(3, 26) # by default, 3-25 m/s in bins of 1 m/s
WS_RANGE = range(3, 26) # this can be set to one value (if the velocity does not affect the resulting wake fields) or a range
TIs = my_arange(0, 1, 0.02) # the ti is percentage
CTs = my_arange(0.1, 24/25, 0.02) #for ct=0, the wake field is not interesting
# TODO change the step size here is decided accordingly to some attempts on the difference between 
# according to the "AeroDyn Theory Manual", the maximum CT should be 24/25 -> see beginning of https://onlinelibrary.wiley.com/doi/full/10.1002/we.2688

In [ ]:
horizontal_grid = get_discretized_grid(TURBINE_DIAMETER,
                                       X_START_FACTOR, X_END_FACTOR, Y_START_FACTOR, Y_END_FACTOR,
                                       STEP_FACTOR)

print(f"Shape of the discretised grid: {len(horizontal_grid.x)}x{len(horizontal_grid.y)}")

for wind_speed in WS_RANGE:
    datasets = list()
    for ti, ct in itertools.product(TIs, CTs):
        print(f"\r{wind_speed=}\t{ti=}\t{ct=}", end="\r")
        site = get_site(ti=ti, ws=wind_speed)
        wind_turbine = get_wind_turbine(TURBINE_DIAMETER, TURBINE_HUB_HEIGHT, TURBINE_POWER_NORM,
                                        constant_ct=ct, ti=ti)
        #plot_ct_curves(wind_turbine)

        # single wake model
        ainslie_model = EddyViscosityModel(site, wind_turbine)

        ds = generate_wake_dataset(ainslie_model, wind_speed, WIND_DIRECTION,
                                    TURBINE_DIAMETER, TURBINE_X, TURBINE_Y,
                                    horizontal_grid, wind_turbine)
        #TODO add control to make sure that I do not add the same input variables twice
        datasets.append(ds)

    print("\nSaving...", end="\r")
    filepath = utils.get_filepath(X_START_FACTOR, X_END_FACTOR, Y_START_FACTOR, Y_END_FACTOR, STEP_FACTOR, wind_speed)
    #final_df = pd.concat(dataframes)
    final_ds = xr.concat([d.stack(z=['x:D', 'y:D', 'ti', 'ct']) for d in datasets], 'z').unstack('z')
    final_ds.to_netcdf(filepath)
    print(f"\rSaved data in '{filepath}'\r")